In [ ]:
import pandas as pd
from pyprojroot import here

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute

import warnings
warnings.filterwarnings('ignore')

In [ ]:
targets_d = pd.read_csv(here() / 'data/preprocessed/targets_d.csv', 
                       parse_dates=True, 
                       index_col='Datetime')

In [ ]:
y = targets_d['Target:Edor_bedoccupying']

In [ ]:
def make_features(y):
    """
    Generate tsfresh features for input vector y
    """
    df = pd.DataFrame(y)
    df['Id'] = 1
    
    rolled = roll_time_series(df, column_id='Id', min_timeshift=7, max_timeshift=7);
    fs = extract_features(rolled, column_id='id', column_sort='sort')
    impute(fs);
    fs.index = y.index[7:]
    filtered = select_features(fs, y.iloc[7:])
    filtered.columns = filtered.columns.str.replace('Target', 'TSF')
    filtered = filtered.round(3)
    return filtered

In [ ]:
mask = targets_d.columns.str.startswith('Target:Edor')
targets = targets_d.loc[:,mask]

feature_list = []

for c in targets.columns:
    y = targets[c]
    features = make_features(y)
    feature_list.append(features)

In [ ]:
features = pd.concat(feature_list, axis=1)

In [ ]:
features.to_csv(here() / 'data/preprocessed/features_d.csv')